In [2]:
from __future__ import print_function
from __future__ import absolute_import
from optparse import OptionParser
import blpapi

def parseCmdLine():
    parser = OptionParser(description="Retrieve realtime data.")
    parser.add_option("-a",
                      "--ip",
                      dest="host",
                      help="server name or IP (default: %default)",
                      metavar="ipAddress",
                      default="localhost")
    parser.add_option("-p",
                      dest="port",
                      type="int",
                      help="server port (default: %default)",
                      metavar="tcpPort",
                      default=8194)
    parser.add_option("--me",
                      dest="maxEvents",
                      type="int",
                      help="stop after this many events (default: %default)",
                      metavar="maxEvents",
                      default=1000000)

    options, _ = parser.parse_args()

    return options




In [3]:
parser = OptionParser(description="Retrieve realtime data.")
parser.add_option("-a",
                    "--ip",
                    dest="host",
                    help="server name or IP (default: %default)",
                    metavar="ipAddress",
                    default="localhost")
parser.add_option("-p",
                    dest="port",
                    type="int",
                    help="server port (default: %default)",
                    metavar="tcpPort",
                    default=8194)
parser.add_option("--me",
                    dest="maxEvents",
                    type="int",
                    help="stop after this many events (default: %default)",
                    metavar="maxEvents",
                    default=1000000)

<Option at 0x197bf287d30: --me>

In [4]:
parser.parse_args()

Usage: ipykernel_launcher.py [options]

ipykernel_launcher.py: error: no such option: --stdin
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\Program Files\Python39\lib\optparse.py", line 1387, in parse_args
    stop = self._process_args(largs, rargs, values)
  File "c:\Program Files\Python39\lib\optparse.py", line 1427, in _process_args
    self._process_long_opt(rargs, values)
  File "c:\Program Files\Python39\lib\optparse.py", line 1480, in _process_long_opt
    opt = self._match_long_opt(opt)
  File "c:\Program Files\Python39\lib\optparse.py", line 1465, in _match_long_opt
    return _match_abbrev(opt, self._long_opt)
  File "c:\Program Files\Python39\lib\optparse.py", line 1670, in _match_abbrev
    raise BadOptionError(s)
optparse.BadOptionError: no such option: --stdin

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\kevinwong\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-in

TypeError: object of type 'NoneType' has no len()

In [ ]:
options = parseCmdLine()

In [ ]:



# Fill SessionOptions
sessionOptions = blpapi.SessionOptions()
sessionOptions.setServerHost(options.host)
sessionOptions.setServerPort(options.port)

print("Connecting to %s:%d" % (options.host, options.port))

# Create a Session
session = blpapi.Session(sessionOptions)

# Start a Session
if not session.start():
    print("Failed to start session.")


if not session.openService("//blp/mktdata"):
    print("Failed to open //blp/mktdata")


security1 = "9696 HK Equity"

subscriptions = blpapi.SubscriptionList()
subscriptions.add(security1,
                    "LAST_PRICE,BID,ASK","",
                    blpapi.CorrelationId(security1))

session.subscribe(subscriptions)

try:
    # Process received events
    eventCount = 0
    while(True):
        # We provide timeout to give the chance to Ctrl+C handling:
        event = session.nextEvent(500)
        for msg in event:
            if event.eventType() == blpapi.Event.SUBSCRIPTION_STATUS or \
                    event.eventType() == blpapi.Event.SUBSCRIPTION_DATA:
                print("%s - %s" % (msg.correlationIds()[0].value(), msg))
            else:
                print(msg)
        if event.eventType() == blpapi.Event.SUBSCRIPTION_DATA:
            eventCount += 1
            if eventCount >= options.maxEvents:
                break
finally:
    # Stop the session
    session.stop()